# Broze To Silver Transformation (Level 1)

This Notebook's purpose is to **transform** the raw data in ***bronze*** conatainer and **store** it onto the ***silver*** container for the further processing.  
In this **Level 1** notebook basic transformation are performed, which includes 
1. Identification & Removal of **Null** values.
2. **Feature Selection** for reduced dataset.
3. Modification of **colomns** names.
4. **Type Casting** of **'date'** colomn.

In [ ]:
dbutils.fs.ls("/mnt/bronze/SalesLT")

In [ ]:
dbutils.fs.ls("/mnt/silver")

In [ ]:
table_names = []

for i in dbutils.fs.ls("/mnt/bronze/SalesLT"):
    table_names.append(i.name.split('/')[0])

In [ ]:
from pyspark.sql.functions import from_utc_timestamp, date_format, col
from pyspark.sql.types import TimestampType

for table in table_names:
    table_path = "/mnt/bronze/SalesLT/" + table + "/" + table + ".parquet"
    df = spark.read.format('parquet').load(table_path)
    columns = df.columns

    for column in columns:
        if 'Date' in column or 'date' in column:
            df = df.withColumn(column, date_format(from_utc_timestamp(col(column).cast(TimestampType()), 'UTC'), 'yyyy-MM-dd'))

    output_path = "/mnt/silver/SalesLT/" + table + "/"
    df.write.format('delta').mode('overwrite').save(output_path)